In [1]:
import os
import math
import pandas as pd
import json
import math
from pyomo.environ import *
import pyomo.environ as pyo

In [2]:
solver = pyo.SolverFactory('gurobi')  # 选择Gurobi求解器

In [3]:
# 读取案例数据
USPE_cost = pd.read_excel('求解数据.xlsx',sheet_name='USPE运费',index_col=0)
USPE_min_cost = pd.read_excel('求解数据.xlsx',sheet_name='USPE最低运价',index_col=0)
Factory2DC = pd.read_excel('求解数据.xlsx',sheet_name='分拨中心',index_col=0)
Order_data = pd.read_excel('求解数据.xlsx',sheet_name='订货量',index_col=0)
Distance_matrix = pd.read_excel(os.path.join('data','全角距离矩阵.xlsx'),index_col=0)
Time_matrix = pd.read_excel(os.path.join('data','全角时间矩阵.xlsx'),index_col=0)

In [4]:
USPE_cost

,Blawnox,City of Industry,Coos Bay,Laughlin,Preston,Tuba City,Vacaville,Walla Walla
Columbus,0.58,0.46,0.46,0.46,0.37,0.46,0.46,0.41
Clancy,0.58,0.46,0.41,0.46,0.37,0.46,0.46,0.41
Missoula,0.58,0.46,0.41,0.46,0.37,0.46,0.46,0.37
Cortez,0.58,0.46,0.46,0.41,0.41,0.37,0.46,0.46
Delta,0.58,0.46,0.46,0.41,0.41,0.37,0.46,0.46
Grand Junction,0.58,0.46,0.46,0.41,0.37,0.37,0.46,0.46
Rifle,0.58,0.46,0.46,0.41,0.41,0.41,0.46,0.46
Wilson,0.58,0.46,0.46,0.41,0.33,0.41,0.46,0.41
Jerome,0.58,0.46,0.41,0.41,0.33,0.41,0.41,0.41
Orofino,0.70,0.46,0.41,0.46,0.41,0.46,0.41,0.33


In [5]:
I = list(USPE_cost.columns[1:])
J = list(USPE_cost.index)
A = {i:Time_matrix.at['Blawnox',i] for i in I}  # A_{i}工厂到DC_{i}的时间
B = {(i,j):Time_matrix.at[i,j] for i in I for j in J}  # B_{ij}DC_{i}到分销商j的时间
C = {j:Time_matrix.at['Blawnox',j] for j in J}
N = {j:Order_data.at[j,'Order Number'] for j in J}
P1 = {i:Factory2DC.at[i,'Truckload Rate'] for i in I}
P2 = {j:USPE_cost.at[j,'Blawnox'] for j in J}
P12 = {(i,j):USPE_cost.at[j,i] for i in I for j in J}
Q = {j:Order_data.at[j,'Weight'] for j in J}
D = {i:Factory2DC.at[i,'Distance from Blawnonx'] for i in I}
H = {i:Factory2DC.at[i,'Handling Cost'] for i in I}
LP2 = {j:USPE_min_cost.at[j,'Blawnox'] for j in J}
LP12 = {(i,j):USPE_min_cost.at[j,i] for i in I for j in J}

In [6]:
B

{('City of Industry', 'Columbus'): 113436,
 ('City of Industry', 'Clancy'): 57950,
 ('City of Industry', 'Missoula'): 61068,
 ('City of Industry', 'Cortez'): 39048,
 ('City of Industry', 'Delta'): 40289,
 ('City of Industry', 'Grand Junction'): 37962,
 ('City of Industry', 'Rifle'): 40748,
 ('City of Industry', 'Wilson'): 136698,
 ('City of Industry', 'Jerome'): 96842,
 ('City of Industry', 'Orofino'): 69414,
 ('City of Industry', 'McCall'): 56557,
 ('City of Industry', 'Boise'): 49382,
 ('City of Industry', 'American Fork'): 32353,
 ('City of Industry', 'Ogden'): 35789,
 ('City of Industry', 'Moab'): 35890,
 ('City of Industry', 'Washington'): 136820,
 ('City of Industry', 'Phoeniz'): 20788,
 ('City of Industry', 'Flagstaff'): 23353,
 ('City of Industry', 'Polacca'): 30055,
 ('City of Industry', 'Prescott Valley'): 24231,
 ('City of Industry', 'Fort Mohave'): 14197,
 ('City of Industry', 'Las Vegas'): 14064,
 ('City of Industry', 'Fernley'): 33564,
 ('City of Industry', 'Reno'): 32014

### 模型1：只考虑成本最优化——2个DC成本最低

In [7]:
def create_model1(I,J,P1,P2,P12,N,Q,D,H,LP2,LP12):
    '''
    I:物流分拨中心编号(1)
    J:分销商编号(2)
    P1_i:工厂到分拨中心i的费率
    P2_j:工厂到分销商j的费率
    P12_ij:分拨中心i到分销商J的费率
    N_j:分销商J的订单数(包裹数)
    Q_j:分销商J的订货量
    T_i:工厂到分拨中心i的运输次数
    D:工厂到分拨中心i的距离
    H_i:分拨中心i的处理费用
    LP2_j:工厂到分销商j的最低运费
    LP12_ij:分拨中心i到分销商j的最低运费
    '''

    model = pyo.ConcreteModel(name='Simplify Model')
    model.m = pyo.Var(I, within=pyo.Binary)
    model.y = pyo.Var(I, J, within=pyo.Binary)
    model.PQ2 = pyo.Var(J, within=pyo.NonNegativeReals)
    model.PQ12 = pyo.Var(I,J, within=pyo.NonNegativeReals)
    model.t = pyo.Var(I, within=pyo.NonNegativeIntegers)

    def obj_rule(md):
        return 40000/52*sum(md.m[i] for i in I) + sum(md.PQ2[j]*(1-sum(md.y[i,j] for i in I)) for j in J) + \
            sum(D[i]*P1[i]*md.t[i]/2 for i in I) + sum(H[i]*md.y[i,j]*N[j] for i in I for j in J) + sum(md.PQ12[i,j]*md.y[i,j] for i in I for j in J)
    model.obj = pyo.Objective(rule=obj_rule)

    def cross_docking_rule(md,i,j):
        return model.m[i] >= model.y[i,j]
    model.cross_docking_rule = pyo.Constraint(I,J, rule=cross_docking_rule)

    def Factory2DC_rule(md,i):
        # if model.y[i,j]*Q[j]/20000 <1:
        #     return model.t[i] == 1
        # if 1 < model.y[i,j]*Q[j]/20000 < 2:
        #     return model.t[i] == 2
        # if 2 < model.y[i,j]*Q[j]/20000 < 3:
        #     return model.t[i] == 3
        return model.t[i] >= sum(model.y[i,j]*Q[j] for j in J)/20000
    model.Factory2DC_rule = pyo.Constraint(I, rule=Factory2DC_rule)

    def PQ2_rule(md,j):
        return md.PQ2[j] == max(P2[j]*Q[j],LP2[j])
    model.PQ2_rule = pyo.Constraint(J, rule=PQ2_rule)

    def PQ12_rule(md,i,j):
        return md.PQ12[i,j] == max(P12[i,j]*Q[j],LP12[i,j])
    model.PQ12_rule = pyo.Constraint(I,J, rule=PQ12_rule)

    def transport_type_rule(md,j):
        return sum(md.y[i,j] for i in I) <= 1
    model.transport_type_rule = pyo.Constraint(J, rule=transport_type_rule)

    # def transport_num_rule(md):
    #     return sum(md.m[i] for i in I) == 1
    # model.dc_num = pyo.Constraint(rule=transport_num_rule)

    return model

model1 = create_model1(I,J,P1,P2,P12,N,Q,D,H,LP2,LP12)

In [8]:
res = solver.solve(model1)

In [9]:
model1.pprint()

18 Set Declarations
    Factory2DC_rule_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {'City of Industry', 'Coos Bay', 'Laughlin', 'Preston', 'Tuba City', 'Vacaville', 'Walla Walla'}
    PQ12_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                    : Size : Members
        None :     2 : PQ12_index_0*PQ12_index_1 :  301 : {('City of Industry', 'Columbus'), ('City of Industry', 'Clancy'), ('City of Industry', 'Missoula'), ('City of Industry', 'Cortez'), ('City of Industry', 'Delta'), ('City of Industry', 'Grand Junction'), ('City of Industry', 'Rifle'), ('City of Industry', 'Wilson'), ('City of Industry', 'Jerome'), ('City of Industry', 'Orofino'), ('City of Industry', 'McCall'), ('City of Industry', 'Boise'), ('City of Industry', 'American Fork'), ('City of Industry', 'Ogden'), ('City of Industry', 'Moab'), ('City of Industry', 'Washington'), ('City of Industry', 'Phoe

In [10]:
md1_y_data = {i: value(v) for i,v in model1.y.items()}
md1_m_data = {i: value(v) for i,v in model1.m.items()}
md1_t_data = {i: value(v) for i,v in model1.t.items()}

md1_y_df = pd.Series(md1_y_data).unstack()
md1_m_df = pd.DataFrame.from_dict(md1_m_data,orient='index',columns=['是否建立转运中心'])
md1_t_df = pd.DataFrame.from_dict(md1_t_data,orient='index',columns=['工厂到转运中心的卡车数'])

In [11]:
md1_y_data

{('City of Industry', 'Columbus'): 0.0,
 ('City of Industry', 'Clancy'): 0.0,
 ('City of Industry', 'Missoula'): 0.0,
 ('City of Industry', 'Cortez'): 0.0,
 ('City of Industry', 'Delta'): 0.0,
 ('City of Industry', 'Grand Junction'): 0.0,
 ('City of Industry', 'Rifle'): 0.0,
 ('City of Industry', 'Wilson'): 0.0,
 ('City of Industry', 'Jerome'): 0.0,
 ('City of Industry', 'Orofino'): 0.0,
 ('City of Industry', 'McCall'): 0.0,
 ('City of Industry', 'Boise'): 0.0,
 ('City of Industry', 'American Fork'): 0.0,
 ('City of Industry', 'Ogden'): 0.0,
 ('City of Industry', 'Moab'): 0.0,
 ('City of Industry', 'Washington'): 0.0,
 ('City of Industry', 'Phoeniz'): 0.0,
 ('City of Industry', 'Flagstaff'): 0.0,
 ('City of Industry', 'Polacca'): 0.0,
 ('City of Industry', 'Prescott Valley'): 0.0,
 ('City of Industry', 'Fort Mohave'): 0.0,
 ('City of Industry', 'Las Vegas'): 0.0,
 ('City of Industry', 'Fernley'): 0.0,
 ('City of Industry', 'Reno'): 0.0,
 ('City of Industry', 'Carlin'): 0.0,
 ('City of 

In [12]:
md1_t_df

,工厂到转运中心的卡车数
City of Industry,-0.0
Coos Bay,-0.0
Laughlin,-0.0
Preston,0.0
Tuba City,1.0
Vacaville,0.0
Walla Walla,1.0


In [88]:
with pd.ExcelWriter(os.path.join('data','最优解(2个仓库)情况.xlsx')) as writer:
	md1_y_df.to_excel(writer, sheet_name="转运方式选择")
	md1_m_df.to_excel(writer, sheet_name="转运中心选择")
	md1_t_df.to_excel(writer, sheet_name="工厂到转运中心次数")

In [89]:
pyo.value(model1.obj)

16954.398538461533

### 模型2：送货时间最优化

In [8]:
def create_model2(I,J,A,B,C,N,Q):
    '''
    I:物流分拨中心编号(1)
    J:分销商编号(2)
    A_i:工厂到DC_i所需时间
    B_{ij}:DC_i到分销商j所需时间
    C_j:工厂到分销商j所需时间
    N_j:分销商J的订单数(包裹数)
    Q_j:分销商J的订货量
    T_i:工厂到分拨中心i的运输次数
    '''

    model = pyo.ConcreteModel(name='Simplify Model')
    model.m = pyo.Var(I, within=pyo.Binary)
    model.y = pyo.Var(I, J, within=pyo.Binary)
    model.t = pyo.Var(I, within=pyo.NonNegativeIntegers)

    def obj_rule(md):
        return sum(md.m[i]*md.t[i]*A[i] for i in I) + sum(N[j]*sum(md.y[i,j]*B[i,j] for i in I) for j in J) + sum((1-md.y[i,j])*C[j]*Q[j] for i in I for j in J) + 302400*(sum(N[j]*md.y[i,j] for i in I for j in J))
    model.obj = pyo.Objective(rule=obj_rule)

    def cross_docking_rule(md,i,j):
        return model.m[i] >= model.y[i,j]
    model.cross_docking_rule = pyo.Constraint(I,J, rule=cross_docking_rule)

    def Factory2DC_rule(md,i):
        return model.t[i] >= sum(model.y[i,j]*Q[j] for j in J)/20000
    model.Factory2DC_rule = pyo.Constraint(I, rule=Factory2DC_rule)

    def transport_type_rule(md,j):
        return sum(md.y[i,j] for i in I) <= 1
    model.transport_type_rule = pyo.Constraint(J, rule=transport_type_rule)

    # 用于限制所建立的仓库数量
    # def transport_num_rule(md):
    #     return sum(md.m[i] for i in I) == 1
    # model.dc_num = pyo.Constraint(rule=transport_num_rule)

    return model

model2 = create_model2(I,J,A,B,C,N,Q)

In [11]:
model2_res = solver.solve(model2)
model2.pprint()

10 Set Declarations
    Factory2DC_rule_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {'City of Industry', 'Coos Bay', 'Laughlin', 'Preston', 'Tuba City', 'Vacaville', 'Walla Walla'}
    cross_docking_rule_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                                                : Size : Members
        None :     2 : cross_docking_rule_index_0*cross_docking_rule_index_1 :  301 : {('City of Industry', 'Columbus'), ('City of Industry', 'Clancy'), ('City of Industry', 'Missoula'), ('City of Industry', 'Cortez'), ('City of Industry', 'Delta'), ('City of Industry', 'Grand Junction'), ('City of Industry', 'Rifle'), ('City of Industry', 'Wilson'), ('City of Industry', 'Jerome'), ('City of Industry', 'Orofino'), ('City of Industry', 'McCall'), ('City of Industry', 'Boise'), ('City of Industry', 'American Fork'), ('City of Industry', 'Ogden'), ('City of Industry', '

In [12]:
# 输出objective function的optimal values
pyo.value(model2.obj)

19282374462.46

In [13]:
md2_y_data = {i: value(v) for i,v in model2.y.items()}
md2_m_data = {i: value(v) for i,v in model2.m.items()}
md2_t_data = {i: value(v) for i,v in model2.t.items()}

md2_y_df = pd.Series(md2_y_data).unstack()
md2_m_df = pd.DataFrame.from_dict(md2_m_data,orient='index',columns=['是否建立转运中心'])
md2_t_df = pd.DataFrame.from_dict(md2_t_data,orient='index',columns=['工厂到转运中心的卡车数'])

In [14]:
with pd.ExcelWriter(os.path.join('data','最优时间情况(考虑拆包时间).xlsx')) as writer:
	md2_y_df.to_excel(writer, sheet_name="转运方式选择")
	md2_m_df.to_excel(writer, sheet_name="转运中心选择")
	md2_t_df.to_excel(writer, sheet_name="工厂到转运中心次数")

In [15]:
md2_y_df

,American Fork,Aurora,Boise,Brewster,Buellton,Carlin,Clancy,Columbus,Cortez,Delta,Eagle Point,Ferndale,Fernley,Flagstaff,Fort Mohave,Fresno,Grand Junction,Jerome,Las Vegas,Los Angeles,McCall,Milton Freewater,Missoula,Moab,Ogden,Orofino,Oxnard,Phoeniz,Polacca,Prescott Valley,Prosser,Reno,Renton,Rifle,Salinas,San Mateo,Santa Clara,Spokane,Susanville,Washington,Westfall,Wilson,Yuba City
City of Industry,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0
Coos Bay,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,1.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0
Laughlin,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0
Preston,1.0,1.0,-0.0,1.0,-0.0,1.0,1.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,1.0,1.0,-0.0
Tuba City,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,1.0,1.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,1.0,1.0,1.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0
Vacaville,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,1.0,1.0,1.0,-0.0,1.0,0.0,-0.0,-0.0,1.0
Walla Walla,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,1.0,1.0,1.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,0.0,-0.0,-0.0,-0.0
